In [735]:
import pandas as pd
import json

# 假设你的JSON文件名为 'data.json'
file_path = './output.json'

# 使用 pandas 的 read_json 方法读取 JSON 文件
df = pd.read_json(file_path)

# 设置显示的最大列数为 None 表示没有限制
pd.set_option('display.max_columns', None)

import re
series = df['网络类型']
data = set()

p = re.compile(r'(;;;(，|游戏运行.{2}|手机性能排行|更多.*?手机＞{0,1}|查看.{2,3}))|>')

# 1. 将所有列表元素用 join 方法 ";;;" 拼接成字符串
# 2. 用正则替换为空
def clean(i):
    if isinstance(i,list):
        i =  ';;;'.join(i)
        i = p.sub("",i)
    return i
df = df.map(clean)
set(df['无线充电'])

# 3. 红外功能非 nan 元素赋值为 "支持"
df.loc[df['红外功能'].notna(), '红外功能'] = '支持'

# 4. NFC非 nan 元素赋值为 "支持"
df.loc[df['NFC'].isna(), 'NFC'] = None
# df.loc[df['NFC'].notna(), 'NFC'] = '支持'
p = re.compile(r'Sumsung Pay|卡模拟|点对点|读卡器|多功能NFC|全方位感应|全功能NFC|全场景')
def clean(x):
    if p.search(x):
        return "支持NFC ({})".format(",".join(p.findall(x))) 
    else:
        return "支持NFC"
df['NFC'] = df.loc[df['NFC'].notna(), 'NFC'].map(clean)

# 5. 无线非 nan 元素赋值为 "多少功率"
p = re.compile(r'(\d{1,2}\.{0,2}\d{0,2})')
def clean(x):
    if p.search(x):
        return "{}W".format(p.search(x).group()) 
    else:
        return "支持"
df['无线充电'] = df.loc[df['无线充电'].notna(), '无线充电'].map(clean)

# 5. 定位导航非 nan 元素赋值为 "哪几种导航技术"
p = re.compile(r'A{0,1}-{0,1}GPS|GPS.{0,3}L1.{0,3}L5.{0,3}|A-{0,1}Glonass|QZSS|Beidou|无线局域网|蜂窝网络定位|北斗|伽利略',flags=re.I)
p2 = re.compile(r';;;|/|，|、|｜|\|')
p3 = re.compile(r' |[\?:（）\(\)：\r\n]|导航|支持|技术|内置|其他|包括|，精度3米|，（L1单频|：|基站|Ip68ip78双3D识别|可|辅助定位|定位|系统|导轨|导|惯性|微')
p4 = re.compile(r'(L1|B1I|（N)([、/，])',flags=re.I)
data = []
def clean(x):
    if isinstance(x,str):
        if p4.search(x):
            x = x.replace(p4.search(x).group(2), "+")
        if "压力感应，重力加速度，陀螺仪/指纹传感器，距离传感器，光敏感应，电子罗盘/红外遥控/线性马达，NFC" in x:
            return "暂无参考信息"
        x = p3.sub("",x)
        x2 = "{}".format(";;;".join(p2.split(x)))
        if ";;;双频;;;" in x2:
            print(x)
        data.extend(p2.split(x2))
        return x2
    else:
        return "暂无参考信息"
df['定位导航'] = df.loc[df['定位导航'].notna(), '定位导航'].map(clean)

df.to_json("output_clean.json", orient="records", force_ascii=False)
set(data)

{'',
 'A-BDS',
 'A-GLONASS',
 'A-GNSS',
 'A-GPS',
 'A-GPSA-BDS',
 'A-GPSA-GLONASS',
 'A-GPSA-Glonass',
 'A-GPSGLONASSG1',
 'A-GPSGalileo',
 'A-GPSGalileoE1+E5a',
 'A-GPSGlonass',
 'A-GPSLTEPP',
 'A-GPS伽利略',
 'A-GPS伽利略E1+E5a双频',
 'A-GPS北斗L1+L5双频',
 'A-GPS双频',
 'A-GPS数据网络',
 'A-GPS无线局域网',
 'A-GPS蜂窝网络',
 'AGNSS',
 'AGPS',
 'AGPS;蜂窝网络;无线局域网',
 'BEIDOU',
 'BEIDOUB1+B2a',
 'BEIDOUB1i++B1Cc+B2a',
 'BeiDou',
 'BeiDouB1I',
 'BeiDouB1i+B1c+B2a',
 'BeiDouB1i+B1c+B2a三频',
 'Beidou',
 'CL5AGNSS',
 'FLONASS',
 'GALILEO',
 'GALILEOE1',
 'GALILEOE1+E5a',
 'GALILEOE1+E5a+E2b三频',
 'GALILEOE1+E5a+E5b',
 'GALILEOE1+E5a+E5b三频',
 'GALILEOE1+E5a+Eb5三频',
 'GALILEOE1+E5a双频',
 'GLONASS',
 'GLONASSE1',
 'GLONASSE1+E5a',
 'GLONASSE1单频',
 'GLONASSG1',
 'GLONASSG1单频',
 'GLONASSGALILEE1+E5a+E5b三频',
 'GLONASSGALILEE1+E5a双频O',
 'GLONASSGALILEO',
 'GLONASSGALILEOQZSS',
 'GLONASSGPSL1+L5双频',
 'GLONASSGalileo',
 'GLONASSG单频1',
 'GLONASSL1',
 'GLONASSLTEEP',
 'GLONASSQZSS',
 'GLONASSiBeacon',
 'GLONASS北斗B1+B2A',
 'GLONASS北

In [599]:
set(df['定位导航'])

{'A-GPS导航，;;;北斗导航Glonass导航，QZSS导航，Galileo导航，;;;GPS导航',
 'A-GPS技术',
 'A-GPS技术;;;北斗导航Glonass导航，QZSS导航，Galileo导航，A-BDS导航，A-GLONASS导航',
 'A-GPS技术A-Glonass导航，Beidou导航，Galileo导航，QZSS导航',
 'A-GPS技术（双频），;;;GLONASS导航;;;北斗导航GALILEO导航',
 'A-GPS，;;;北斗导航Glonass导航，QZSS导航，Galileo导航，;;;GPS导航无线局域网，蜂窝网络定位',
 'AGPS定位，蜂窝网络定位，无线局域网定位',
 'Beidou导航，;;;GLONASS导航;;;GPS导航AGPS导航',
 'Beidou导航，Galileo导航，;;;GLONASS导航;;;GPS导航A-GPS，QZSS导航',
 'Beidou导航，Galileo导航，;;;GLONASS导航;;;GPS导航AGPS导航',
 'GLONASS导航GPS导航（L1+L5双频），Galileo导航（E1+E5a双频），北斗导航（B1I+B1C+B2a三频），QZSS导航（L1+L5双频），AGPS技术，无线局域网，蜂窝网络定位，电子罗盘',
 'GNSS GPS导航(L1/L5双频)， Glonass导航(G1单频)， Galileo导航(E1/E5a双频)， BeiDou导航(B1i/B1c/B2a三频)， QZSS导航(L1/L5双频)  NavIC导航(=IRNSS单频)',
 'GPS',
 'GPS (L1 + L5 双频) / AGPS / GLONASS / 北斗 ((B1I+B1C+B2a+B2b 四频) / GALILEO (E1 + E5a + E5b 三频) / QZSS (L1 + L5 双频) / NavIC',
 'GPS (L1 + L5 双频) ，;;;A-GPS技术;;;GLONASS导航北斗导航 (B1I + B1C + B2a 三频)，GALILEO导航 (E1 + E5a 双频)，QZSS导航 (L1 + L5 双频)，NavIC导航',
 'GPS (L1+L5 双频) ，AGPS，GLONASS，北斗 (B1I+B1C+B2a+B2b 四